In [160]:
# imports a library 'pandas', names it as 'pd'

import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV


# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [161]:
read_in_file = pd.read_csv('C:/Users/smnio/Desktop/projects/local_only/appt1.csv')#, nrows=1000000)



In [162]:
#%reset 

In [163]:
read_in_file=read_in_file.fillna(0)


In [164]:
read_in_file = read_in_file.dropna()
df= read_in_file.copy()
df.head()

,﻿app_month,appt_begintime,weekday,days_made_in_advance,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,...,Dental- Initial Dental Assessm,Annual-Adult Male,Dental- Medium,HB Renewal,Optometry Dispensing,FP-Routine Visit,Dental- Emergency Visit,Dental- Comprehensive 60 Min,BH - BHI Counseling 45 Min,Optometry Medically Necessary
0,1,1,2,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,3,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
cols = ['month']
for i in range(1,len(df.columns)):
    cols.append(df.columns[i])
df.columns=cols

df = pd.concat([df, pd.get_dummies(df['month'], prefix='month')], axis=1)

In [166]:
df = pd.concat([df, pd.get_dummies(df['weekday'], prefix='wkday')], axis=1)

In [167]:
df = pd.concat([df, pd.get_dummies(df['appt_begintime'], prefix='hour')], axis=1)

In [168]:
df.income = df.income.round(-4)
df = pd.concat([df, pd.get_dummies(df['income'], prefix='inc')], axis=1)

In [169]:
#break out days appoitnment was made in advance into different buckets
df['appt_made_same_day'] = 0
df['appt_made_previous_day'] = 0
df['appt_made_2_6_days_prior'] = 0
df['appt_made_1_week_prior'] = 0
df['appt_made_2_week_prior'] = 0
df['appt_made_3_week_prior'] = 0
df['appt_made_1_6_months_prior'] = 0
df['appt_made_over_half_year'] = 0


df['appt_made_same_day'] = df.days_made_in_advance.map(lambda x: 1 if (x == 0) else 0)
df['appt_made_previous_day'] = df.days_made_in_advance.map(lambda x: 1 if (x == 1) else 0)
df['appt_made_2_6_days_prior'] =  df.days_made_in_advance.map(lambda x: 1 if (x > 1 and x < 8) else 0)
df['appt_made_1_week_prior']  =  df.days_made_in_advance.map(lambda x: 1 if (x > 7 and x < 15) else 0)
df['appt_made_2_week_prior']  =  df.days_made_in_advance.map(lambda x: 1 if (x > 14 and x < 22) else 0)
df['appt_made_3_week_prior']  =  df.days_made_in_advance.map(lambda x: 1 if (x > 21 and x < 29) else 0)
df['appt_made_1_6_months_prior']  =  df.days_made_in_advance.map(lambda x: 1 if (x > 30 and x < 182) else 0)
df['appt_made_over_half_year']  =  df.days_made_in_advance.map(lambda x: 1 if (x > 181) else 0)

df.head()

,month,appt_begintime,weekday,days_made_in_advance,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,...,inc_210000,inc_220000,appt_made_same_day,appt_made_previous_day,appt_made_2_6_days_prior,appt_made_1_week_prior,appt_made_2_week_prior,appt_made_3_week_prior,appt_made_1_6_months_prior,appt_made_over_half_year
0,1,1,2,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,1,1,2,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,1,2,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,1,2,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,1,3,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [170]:
#break out ages into discrete groups

df['age_65_and_over'] = 0
df['age_under_5'] = 0
df['age_5_17'] = 0
df['age_18_29'] = 0
df['age_30_44'] = 0
df['age_45_64'] = 0


df['age_65_and_over'] = df.age.map(lambda x: 1 if (x > 64) else 0)
df['age_under_5'] = df.age.map(lambda x: 1 if (x < 5) else 0)
df['age_5_17'] = df.age.map(lambda x: 1 if (x > 4 and x <18 ) else 0)
df['age_18_29'] = df.age.map(lambda x: 1 if (x > 17 and x <30 ) else 0)
df['age_30_44'] = df.age.map(lambda x: 1 if (x > 29 and x <45 ) else 0)
df['age_45_64'] = df.age.map(lambda x: 1 if (x > 44 and x <65 ) else 0)

df.head()

,month,appt_begintime,weekday,days_made_in_advance,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,...,appt_made_2_week_prior,appt_made_3_week_prior,appt_made_1_6_months_prior,appt_made_over_half_year,age_65_and_over,age_under_5,age_5_17,age_18_29,age_30_44,age_45_64
0,1,1,2,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,1,2,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,1,2,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1,1,3,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [171]:
columnNames=df.columns

In [172]:
from sklearn import preprocessing

xi = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
xi_scaled = min_max_scaler.fit_transform(xi)
df = pd.DataFrame(xi_scaled)


df.columns=columnNames



In [173]:
df.corr()

,month,appt_begintime,weekday,days_made_in_advance,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,...,appt_made_2_week_prior,appt_made_3_week_prior,appt_made_1_6_months_prior,appt_made_over_half_year,age_65_and_over,age_under_5,age_5_17,age_18_29,age_30_44,age_45_64
month,1.000000,0.006495,-0.029236,0.061517,0.034680,-0.025413,-0.020633,0.029918,-0.023536,-0.016457,...,0.027559,0.021953,0.042908,0.010397,0.008194,0.005094,0.011782,0.004195,-0.016131,-0.004073
appt_begintime,0.006495,1.000000,-0.072311,0.021107,-0.029517,0.013192,0.024015,-0.044848,0.024315,0.033361,...,0.018837,0.004478,0.012312,0.001614,-0.077180,0.000776,0.104795,0.046479,0.017399,-0.026403
weekday,-0.029236,-0.072311,1.000000,-0.009358,0.005648,-0.004644,-0.002974,-0.010627,0.006251,0.007517,...,0.004856,-0.001763,-0.009109,-0.000341,-0.010596,0.001746,0.018841,0.009601,-0.005527,-0.001780
days_made_in_advance,0.061517,0.021107,-0.009358,1.000000,-0.162752,0.156780,0.068139,-0.027730,0.031094,0.007899,...,0.168823,0.197970,0.803461,0.308971,-0.047628,-0.003403,0.011430,0.016197,0.038309,-0.004237
kept,0.034680,-0.029517,0.005648,-0.162752,1.000000,-0.520505,-0.757326,0.111109,-0.065647,-0.078363,...,-0.065505,-0.060967,-0.121926,-0.013518,0.048165,0.032058,0.004120,-0.024059,-0.016894,-0.026199
rescheduled,-0.025413,0.013192,-0.004644,0.156780,-0.520505,1.000000,-0.163410,-0.062461,0.116400,-0.018949,...,0.069012,0.058847,0.112414,0.014293,-0.022641,-0.009595,0.003522,-0.014639,0.002042,0.030742
cancelled,-0.020633,0.024015,-0.002974,0.068139,-0.757326,-0.163410,1.000000,-0.080606,-0.013174,0.105034,...,0.022904,0.025435,0.054899,0.004688,-0.038335,-0.029702,-0.007453,0.038994,0.017957,0.006759
last_kept,0.029918,-0.044848,-0.010627,-0.027730,0.111109,-0.062461,-0.080606,1.000000,-0.540432,-0.745232,...,-0.010737,-0.008232,-0.023724,0.000452,0.053197,0.026124,0.001128,-0.021754,-0.017339,-0.028629
last_rescheduled,-0.023536,0.024315,0.006251,0.031094,-0.065647,0.116400,-0.013174,-0.540432,1.000000,-0.158295,...,0.020026,0.010784,0.021328,-0.000202,-0.026982,-0.009138,0.003053,-0.010356,0.003543,0.030597
last_cancelled,-0.016457,0.033361,0.007517,0.007899,-0.078363,-0.018949,0.105034,-0.745232,-0.158295,1.000000,...,-0.003270,0.001114,0.010938,-0.000371,-0.041045,-0.023416,-0.003744,0.033735,0.017541,0.009348


In [174]:
#df.corr()['kept'].order(ascending=False)

In [175]:
#k=corrVals[corrVals.ix[:1]>.001]
#print(k)

In [176]:
corr_series = df.corr()['kept'].order(ascending=False)
#pd.set_option('display.height', 500)
#pd.set_option('display.max_rows', 500)

print(corr_series)

kept                             1.000000
appt_made_same_day               0.241160
kept_perc                        0.189673
last_kept                        0.111109
income                           0.065824
age_65_and_over                  0.048165
lang_Eng                         0.036791
month                            0.034680
age_under_5                      0.032058
age                              0.025966
inc_120000                       0.025547
inc_100000                       0.024164
male                             0.022561
inc_90000                        0.021977
Annual-Adult Female              0.018765
inc_110000                       0.018363
HB Consultation                  0.016597
hour_5                           0.015711
hour_2                           0.014645
inc_130000                       0.014565
FP-Routine Visit                 0.014516
eth_refused                      0.014454
hour_3                           0.014240
eth_not_hispanic                 0

In [177]:
selected_columns = corr_series[((corr_series > .02 ) | (corr_series < -.02))].index

selected_columns = selected_columns[((selected_columns != 'kept' ) & (selected_columns != 'cancelled') & (selected_columns != 'rescheduled'))]

print(selected_columns)

Index([u'appt_made_same_day', u'kept_perc', u'last_kept', u'income',
       u'age_65_and_over', u'lang_Eng', u'month', u'age_under_5', u'age',
       u'inc_120000', u'inc_100000', u'male', u'inc_90000', u'hour_17',
       u'female', u'age_18_29', u'inc_30000', u'age_45_64', u'inc_40000',
       u'appt_begintime', u'inc_50000', u'lang_Chi', u'month_2',
       u'appt_made_3_week_prior', u'appt_made_2_week_prior',
       u'last_rescheduled', u'appt_made_1_week_prior', u'last_cancelled',
       u'appt_made_2_6_days_prior', u'appt_made_previous_day', u'resch_perc',
       u'appt_made_1_6_months_prior', u'can_perc', u'days_made_in_advance'],
      dtype='object')


In [178]:

X = df.ix[:, selected_columns] 

y = df.ix[:, df.columns == 'kept']




In [179]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [180]:


#np.all(np.isfinite(mat))

In [181]:
#supervised classification
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()

#lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

C:\Users\smnio\Anaconda\lib\site-packages\sklearn\utils\validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.74022207490884984

In [182]:
#df.info()#0.71724745080587426

In [183]:
read_in_file = read_in_file.dropna()



y_pred = lr.predict(X)



data = pd.DataFrame({"Actual":read_in_file.kept, "Predicted":y_pred.tolist()})



In [184]:
#get calculated coeffcicients
lr.coef_

array([[ 1.72845545, -0.67269075,  0.24323726,  0.35656358,  0.14888946,
        -0.64001428,  0.25552979,  0.42549612, -0.28239894,  0.0135947 ,
        -0.09113519,  0.23482862, -0.01845504, -0.11078536,  0.17345286,
        -0.07999128, -0.14313729, -0.02236122, -0.25653945, -0.22607378,
        -0.10438664, -0.77386129, -0.37010574, -0.03622717,  0.08393132,
         0.23414561,  0.37716183,  0.1870713 ,  0.14836601, -0.49163983,
        -2.16580187, -0.04727266, -2.07058152, -1.92308715]])

In [185]:
#import csv

#data_top=data.head(10000)

#data_top.to_csv("predictions.csv")

